# Installation and Repository cloning

In [ ]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#!pip install adversarial-robustness-toolbox -U
#!pip install git+https://github.com/RobustBench/robustbench.git
#!pip install multiprocess
#!pip install matplotlib
#!pip install pillow
#!pip install foolbox

## Import Libraries

In [1]:
import torch
import utils
import adversarial_attack.attack_utils as attack_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\.venv\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\.venv\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\.venv\lib\site-

cuda


## Load and Prepare Adversarial Model

In [2]:
model = 'CroceL1' # MainiAVG, MainiMSD, CroceL1, Wang2023Better_WRN-28-10, Wang2023Better_WRN-28-10
net, art_net, fb_net, alias = utils.get_model(modelname=model, norm='L2') # specify the norm in case of robustbench models

no input normalization


c:\Users\Admin\Desktop\Python\elastic_net_adversarial_attack\utils.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('./models/pretrained_models/CroceL1

## Load the dataset

In [3]:
splitsize = 50 # full, int: splitsize
xtest, ytest = utils.load_dataset(dataset_split=splitsize)

Files already downloaded and verified


## Test Accuracy

In [4]:
utils.test_accuracy(net, xtest, ytest)


Accuracy of the test set is: 78.000%



## Parameters 

In [ ]:
Experiment = attack_utils.Experiment_class(art_net, fb_net, net, xtest, ytest, alias,
                        epsilon_l1 = 12, 
                        epsilon_l2= 0.5, 
                        eps_iter = 0.15,  
                        norm = 1,  # 1, 2, np.inf
                        max_iterations_fast_attacks = 10, 
                        max_iterations_slow_attacks = 10, 
                        verbose=True)

Hyperparameter Sweep Experiment

In [9]:
#hyperparameter sweep. Pick only one model.
attack_type = 'exp_attack'
hyperparameter_range=[0.1,0.2,0.5,1.0,2.0]
hyperparameter = 'learning_rate' #'beta'

#results_dict_hyperparameter_sweep = Experiment.hyperparameter_sweep(hyperparameter=hyperparameter, range=hyperparameter_range, attack_type=attack_type)

In [ ]:
#Attack comparison
attack_types = [#'fast_gradient_method', #bounded-full
                #'projected_gradient_descent', #bounded-full
                #'pgd_early_stopping', #not-bounded
                #'auto_projected_gradient_descent', #bounded-full
                #'deep_fool', #not-bounded
                #'brendel_bethge', #bounded-min
                #'carlini_wagner_l2', #not-bounded
                'elastic_net', #not-bounded
                'exp_attack',
                #'exp_attack_smooth',
                #'elastic_net_L1_rule', #not-bounded
                #'elastic_net_L1_rule_higher_beta', #not-bounded
                #'ART_AutoAttack', #bounded-full
                #'original_AutoAttack', #bounded-full
                ]

results_dict_attack_comparison = Experiment.attack_comparison(attack_types)

		-------------------------- Processing Attack: elastic_net --------------------------

20 images done. Current Adversarial Accuracy (L1 / L2): 65.0 / 60.0%
